In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false
}

<IPython.core.display.Javascript object>

In [15]:
from CoreConfig.CoreConfig import CoreConfig
from DBUtility.DBUtility import DBConnection, CreateEndorsementTableIfNotExist
from DBUtility.DBUtility import FetchEndorsementRecordAll, FetchEndorsementRecordRecent
from DBUtility.DBUtility import InsertEndorsementRecord
from pandas import DataFrame
import datetime


import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, interactive_output
from IPython.display import display_pretty
import pandas as pd
from ipywidgets import Layout, Button, Box
from StylingUtility.StylingUtility import CustomStyling

# CoreConfig.initialize_core_config()
# DBSQLiteConnection().configure_connection_information(**CoreConfig().get_sqlite3_db_connection_info())
# CreateEventsTableIfNotExist().execute()

### Endorsement Form - Insert

In [20]:
# get configuration details
CoreConfig.initialize_core_config()

# establish database connection
DBConnection().configure_connection_information(**CoreConfig().get_db_connection_info())

# create table if not exist
CreateEndorsementTableIfNotExist().execute()

Initialize nab endorsement table failed. Please check your connection
Relation "nab_partnership_endorsed_initiatives" already exists



In [10]:
FetchEndorsementRecordAll().execute()

[]

In [38]:
title = widgets.HTML("<center><H3>Recent Entries</H3><center>")

formtitle = widgets.HTML("<center><H3>NAB Partnership – Initiatives Creation Form</H3><center>")

df_out = widgets.Output(layout={'border': '1px solid black', 'aligh_items' : 'center'})

with df_out:
    df = DataFrame(FetchEndorsementRecordRecent().execute(), columns=['initiative_name', 'initiative_short_description', 'initiative_category', 'creation_date', 'type', 'stakeholder', 'team_recommendation', 'endorsement_execution', 'endorsement_execution_date', 'endorsement_body', 'system_creation_date'])
    display(df)


# date widgets
creation_date = widgets.DatePicker(
                value = datetime.datetime.now().date(),
                description='Initiave Creation Date',
                layout=CustomStyling.box_layout,
                style = CustomStyling.style,
                disable=False)

endorsement_execution_date = widgets.DatePicker(
                            value = datetime.datetime.now().date(),
                            description='Endorsement / Execution Date',
                            layout=CustomStyling.box_layout,
                            style = CustomStyling.style,
                            disable=False)

# text widgets
initiative_name = widgets.Text(
                  value='',
                  placeholder='Type Here',
                  description='Initiative Name',
                  layout=CustomStyling.box_layout,
                   style=CustomStyling.style,
                  disable=False)

initiative_short_description = widgets.Text(
                              value='',
                              placeholder='Type Here',
                              description='Initiative Short Description',
                              layout=CustomStyling.box_layout,
                               style=CustomStyling.style,
                              disable=False)

stakeholder = widgets.Text(
                  value='',
                  placeholder='Type Here',
                  description='Stakeholder',
                  layout=CustomStyling.box_layout,
                   style=CustomStyling.style,
                  disable=False)

endorsement_body = widgets.Text(
                  value='',
                  placeholder='Type Here',
                  description='Endorsement Body',
                  layout=CustomStyling.box_layout,
                   style=CustomStyling.style,
                  disable=False)


# dropdown    
initiative_category = widgets.Dropdown(options= {'Open Banking': 'Open Banking',
                                                 'Ecosystems': 'Ecosystems',
                                                 'Other': 'Other'},
                                       value = 'Ecosystems',
                                       description='Initiative Category',
                                       layout=CustomStyling.box_layout,
                                       style=CustomStyling.style)

type = widgets.Dropdown(options= {'Strategy': 'Strategy',
                                     'Business Development': 'Business Development',
                                     'Execution': 'Execution'},
                                       value = 'Strategy',
                                       description='Type',
                                       layout=CustomStyling.box_layout,
                                       style=CustomStyling.style)

def on_type_value_change(change):
#     print(change['owner'])
    if change['owner'].value == 'Strategy' or change['owner'].value == 'Business Development':
        team_recommendation.disabled=False
    else:
        team_recommendation.disabled=True
type.observe(on_type_value_change, names='value')


team_recommendation = widgets.Dropdown(options= {'Go': 'Go',
                                     'No Go': 'No Go'},
                                       value = 'Go',
                                       description='Team Recommendation',
                                       layout=CustomStyling.box_layout,
                                       style=CustomStyling.style)

# radio button
endorsement_execution = widgets.RadioButtons(
                        options=['Y', 'N'],
                        value='Y',
                        description='Endorsement / Execution',
                        disabled=False,
                        style=CustomStyling.style
                    )

# button
button = widgets.Button(
           description='Submit the form',
           disable=False,
           button_style ='', # success
           tooltip='Click me',
           icon='check'
           )

def on_submit_form_button_clicked(b):
    outputs = { 'initiative_name': [initiative_name.value],
                'initiative_short_description': [initiative_short_description.value],
                'initiative_category': [initiative_category.value],
                'creation_date': [creation_date.value],
                'type': [type.value],
                'stakeholder': [stakeholder.value],
                'team_recommendation': [team_recommendation.value],
                'endorsement_execution': [endorsement_execution.value],
                'endorsement_execution_date': [endorsement_execution_date.value],
                'endorsement_body': [endorsement_body.value]
               }
    InsertEndorsementRecord(initiative_name.value, initiative_short_description.value, initiative_category.value, creation_date.value, type.value, stakeholder.value, team_recommendation.value, endorsement_execution.value, endorsement_execution_date.value, endorsement_body.value).execute()
    df = DataFrame(FetchEndorsementRecordRecent().execute(), columns=['initiative_name', 'initiative_short_description', 'initiative_category', 'creation_date', 'type', 'stakeholder', 'team_recommendation', 'endorsement_execution', 'endorsement_execution_date', 'endorsement_body', 'system_creation_date'])
    df_out.clear_output()
    with df_out:
        display(df)
        
button.on_click(on_submit_form_button_clicked)






# user interface organised using VBox (vertical)
ui = widgets.VBox(children=[title,
                            df_out,
                            formtitle,
                            initiative_name,
                            initiative_short_description,
                            initiative_category,
                            creation_date,
                            type,
                            stakeholder,
                            team_recommendation,
                            endorsement_execution,
                            endorsement_execution_date,
                            endorsement_body,
                            button], layout=CustomStyling.items_layout)

# live display of values, for debugging purposes
def form(initiative_name, initiative_short_description, initiative_category, creation_date, type, stakeholder, team_recommendation, endorsement_execution, endorsement_execution_date, endorsement_body):
#     display(initiative_name, initiative_short_description, initiative_category, creation_date, type, stakeholder, team_recommendation, endorsement_execution, endorsement_execution_date, endorsement_body)
    return initiative_name, initiative_short_description, initiative_category, creation_date, type, stakeholder, team_recommendation, endorsement_execution, endorsement_execution_date, endorsement_body

out = interactive_output(form, {'initiative_name': initiative_name,
                                    'initiative_short_description': initiative_short_description,
                                    'initiative_category': initiative_category,
                                    'creation_date': creation_date,
                                    'type': type,
                                    'stakeholder': stakeholder,
                                    'team_recommendation': team_recommendation,
                                    'endorsement_execution': endorsement_execution,
                                    'endorsement_execution_date': endorsement_execution_date,
                                    'endorsement_body': endorsement_body
                               })

display(ui, out)

Output()

In [36]:
print(type.value)

Business Development
